In [16]:
pip install tweepy==3.10.0

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Import the necessary packages
import tweepy
from tweepy.auth import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json

In [9]:
consumer_key    = 'KqVfTgSOfbl0o5wtbSRZa5JNf'
consumer_secret = 'GPCwOovDEhZNk3ZromvnCPESMHxz84R6mhPPCWmnqqR85Ejzf0'
access_token    = '1482336879676698635-NS4yiqQrv9CnBAjNTg0HXTKOAXp3W0'
access_secret   = 'xn5r9K9853mHS02bvNtiDAK3rcClUzUCh24yMMAylVsMgpMlYE'


In [10]:
#Create a StreamListener instance
# On crée une classe qui hérite de la classe StreamListner 
#TweetListener represents an instance of StreamListener that connects to the Twitter API and returns a tweet at a time.
class TweetsListener(StreamListener):

    def __init__(self, csocket):
        self.client_socket = csocket
    # we override the on_data() function in StreamListener
    def on_data(self, data):
        try:
            message = json.loads( data )
            print( message['text'].encode('utf-8') )
            self.client_socket.send( message['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def if_error(self, status):
        print(status)
        return True

In [11]:
#Send data from Twitter
#In order to retrieve data from Twitter API, we must first authenticate our connection using the pre-defined credentials. 
def send_tweets(c_socket):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=['Parkinson']) #on est interssé à la maladie Parkinson

In [13]:
#Start streaming
#Streaming the data from the Twitter API 
#requires creating a listening TCP socket in the local machine (server) 
#on a predefined local IP address and port. 
#A socket consists of a server-side, which is our local machine, 
#and a client-side, which is Twitter API. 
#The open socket from the server-side listens 
#for connections from the client. 
#When the client-side is up and running, 
#the socket will receive the data from the Twitter API based on 
#the topic or keywords defined in the StreamListener instance.

In [15]:
if __name__ == "__main__":
    new_skt = socket.socket()             # initiate a socket object
    host = "127.0.0.1"                    # adresse de la machine locale
    port = 2000                           # on spécifié le port de nootre service
    new_skt.bind((host, port))            #on fait le binding entre notre host et port

    print("Now listening on port: %s" % str(port))

    new_skt.listen(5)                     #  waiting for client connection.
    c, addr = new_skt.accept()            # Establish connection with client. 
                                          # it returns first a socket object,c, and the address bound to the socket

    print("Received request from: " + str(addr))
    # and after accepting the connection, we aill sent the tweets through the socket
    send_tweets(c)

Now listening on port: 2000
Received request from: ('127.0.0.1', 52973)
